# This code clusters and visualizes traffic fatalities in New York City from 2012 to 2016

Import packages and the file as dataframe

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv('SUPER_CLEANED_NYPD_TRAFFIC_COLLISION.csv')

In the next cell we create a dictionary corresponding to the fatalities. The dictionary contains the different years from 2012 to 2016 as keys and the corresponding values are nested lists with the latitude and longitude values

In [ ]:
#The list on 0 index in each year is the Latitude
#The list on 1 index in each year is the Longitude
FatalityYear={2012:[[],[]], 2013:[[],[]],2014:[[],[]], 2015:[[],[]], 2016:[[],[]], 2017:[[],[]]}
for index, row in df.iterrows():
    b=int(row['NUMBER OF PERSONS KILLED'])
    if (b>0) and not(np.isnan(row['LATITUDE']) and np.isnan(row['LONGITUDE'])):
        if int(pd.to_datetime(row['DATE']).year)==2016:
            FatalityYear[2016][0].append(float(row['LATITUDE']))
            FatalityYear[2016][1].append(float(row['LONGITUDE']))
        elif int(pd.to_datetime(row['DATE']).year)==2015:
            FatalityYear[2015][0].append(float(row['LATITUDE']))
            FatalityYear[2015][1].append(float(row['LONGITUDE']))
        elif int(pd.to_datetime(row['DATE']).year)==2014:
            FatalityYear[2014][0].append(float(row['LATITUDE']))
            FatalityYear[2014][1].append(float(row['LONGITUDE']))
        elif int(pd.to_datetime(row['DATE']).year)==2013:
            FatalityYear[2013][0].append(float(row['LATITUDE']))
            FatalityYear[2013][1].append(float(row['LONGITUDE']))
        elif int(pd.to_datetime(row['DATE']).year)==2012:
            FatalityYear[2012][0].append(float(row['LATITUDE']))
            FatalityYear[2012][1].append(float(row['LONGITUDE']))

We cluster the fatalities for different years and visualize them in the next cell using the DBSCAN algorithm
The DBSCAN library is imported from the sklearn.cluster package

In [ ]:
def dbscan_clustering():

    import pandas as pd, numpy as np, matplotlib.pyplot as plt, time
    from sklearn.cluster import DBSCAN
    from sklearn import metrics
    from geopy.distance import great_circle
    import datetime

    year = input("What year do you want to cluster? (2012, 2013, 2014, 2015, 2016): ")
    year = int(year)

    final_list=[]
    for j in range(len(FatalityYear[year][0])):
        my_list=[]
        for i in range(len(FatalityYear[year])):
            my_list.append(FatalityYear[year][i][j])
        final_list.append(my_list)


    coords = np.array(np.radians(np.array(final_list)))

    #The DBSCAN clustering part starts from here
    #I will be taking a radius of 2kms and a minimum of 3 fatalities





    kms_per_radian = 6371.0088
    epsilon = 2 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=3, algorithm='ball_tree', metric='haversine').fit(coords)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    #db.labels_ is a list containing the labels for all the points (118 in this case) as to which cluster they belong to
    #The labels are from 0 to the total number of clusters. The outliers are given a label of -1
    #np.zeros_like() gives a list of zeros with the same shape of the list mentioned (db.labels_ in this case)
    #The additional parameter dtype=bool converts each value to False since all of them are zeros
    core_samples_mask[db.core_sample_indices_] = True
    #db.core_sample_indices_ gives all the indices of db at which the values are not -1
    #core_samples_mask[db.core_sample_indices_] = True converts all values that were False at these indices to True
    #We now have core_samples_mask which is a list containing boolean values with True at points belonging to a cluster
    #and False at points that are outliers
    labels = db.labels_
    #We then save all the labels in a variable called labels
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    #The number of clusters is stored in n_clusters_ 
    #Note that the -1 is not a cluster and hence is not counted in this

    print('Estimated number of clusters: %d' % n_clusters_)

    #forming a temporary list to store all labels and label -1 as well for outliers if present to plot outliers as well
    temp_labels=[]
    for i in range(n_clusters_):
        temp_labels.append(i)
    if -1 in labels:
        temp_labels.append(-1)

    clusters = pd.Series([coords[labels == n] for n in temp_labels])#range(n_clusters_)])


    main_list=[]
    for i in range(n_clusters_+1):
        Lat_list=[]
        Long_list=[]
        for j in range(len(clusters[i])):
            Lat_list.append(FatalityYear[year][0][np.where(coords==[clusters[i][j]])[0][0]])
            Long_list.append(FatalityYear[year][1][np.where(coords==[clusters[i][j]])[0][0]])
            #np.where(coords==[clusters[i][j]]) provides us with a tuple containing two arrays like this
            #(array([65, 65], dtype=int64), array([0, 1], dtype=int64))
            #The first array contains the index of the list coords at which the jth radian coordinates of the ith cluster is.
            #The next two [0][0] let us reach that index in FatalityYear[year][lat/long]
        main_list.append([Long_list,Lat_list])
        #Add these lists of lat and long in a list for each cluster and finally add them to a main_list


    from mpl_toolkits.basemap import Basemap

    m= Basemap(projection='mill', llcrnrlat = 40.471920, llcrnrlon = -74.297241,
                          urcrnrlat=40.938311,
                          urcrnrlon = -73.635315, epsg=2263, resolution = 'h')
    unique_labels = list(set(labels))
    colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))
    m.arcgisimage(service='ESRI_StreetMap_World_2D', xpixels = 12000, verbose= True)
    m.drawcoastlines()
    m.drawcountries(linewidth=2)
    #m.drawstates(linewidth=2)
    m.drawcounties()
    m.drawrivers()

    for i,col in zip(range(len(main_list)), colors):
        if unique_labels[i] == -1:
            # Black used for noise.
            col = 'k'
        xpt, ypt = m(main_list[i][0],main_list[i][1])
        m.plot(xpt, ypt, 'co', markersize =9,markerfacecolor=col)
    title = ("Cluster Visualization for " + str(year))
    plt.title(title)
    plt.savefig(("Fatality Cluster Visualization for year " + str(year) +'.png'), dpi=1200)
    plt.show()
    answer=input("Do you want to cluster more? (Y/N): ")
    if answer=='Y':
        dbscan_clustering()


dbscan_clustering()